# Tarea 7. Experimentacion

Esta tarea consiste en llevar a cabo la experimentación de diversos
clasificadores sobre una variedad de datasets. Se reportarán los resultados y,
posteriormente cuando tengamos la clase de significancia estadística, haremos el
análisis de los clasificadores y si hay un ganador cuál de ellos sería.

Las instrucciones se detallan a continuación:

1. Buscar varios datasets (7, 8, 10, etc.) que cumplan con una condición: deben
   tener alguna característica en común. ¿Qué característica en común? Hay
   varias cosas que pueden satisfacer esta condición, por ejemplo, que traten el
   mismo problema (e.g. diagnósticos médicos aunque sean de diferente
   enfermedad), que tengan muchas más variables que instancias (maldición de la
   dimensionalidad), que todos sean de más de 3 clases, que sus clases estén
   desbalanceadas, entre otras cosas que ustedes puedan identificar. La idea es
   que dichos datasets puedan ser identificados en el mismo contexto.

2. Llevar a cabo la clasificación, utilizando scikit-learn, aplicando los
   clasificadores que hemos visto y algunos otros de su predilección. Para
   realizar la clasificación, deben emplear algún método de validación cruzada
   (k-fold cross-validation, o leave-one-out, dependiendo la cantidad de datos).
   Asimismo, deberán reportar los resultados utilizando el balanced accuracy,
   sensibilidad y especificidad (juntas), o bien el área bajo la curva roc (AUC
   ROC).

3. Posteriormente realizaremos alguna prueba de significancia estadística (cuyo
   tema veremos al regreso de vacaciones), con la finalidad de conocer si
   existen diferencias estadísticamente significativas entre los clasificadores.



# Datos de medidas morfológicas de animales para inferir especie o sexo.

1. [Abalone dataset](https://archive-beta.ics.uci.edu/dataset/1/abalone)

1. [Birds
   bones dataset](https://www.kaggle.com/datasets/zhangjuefei/birds-bones-and-living-habits)

1. [Penguins dataset](https://archive-beta.ics.uci.edu/dataset/690/palmer+penguins-3)

1. [Pokemon_dataset](https://www.kaggle.com/datasets/cristobalmitchell/pokedex)

1. [Sloths dataset](https://www.kaggle.com/datasets/bertiemackie/sloth-species)

In [ ]:
import pandas as pd

abalones = pd.read_csv("../datasets/abalone.csv")
print(abalones.shape)
abalones_ft = abalones.loc[:,['Shell_length', 'Shell_diameter', 'Height', 'Whole_weight',
       'Shucked_weight', 'Viscera_weight', 'Shell_weight',]]
abalones_tg = abalones[['Sex_category']]

birds = pd.read_csv("../datasets/bird.csv")
print(birds.shape)
birds_ft = birds.loc[:,['Humerus_length', 'Humerus_diameter', 'Ulna_length',
       'Ulna_diamater', 'Femur_length', 'Femur_diameter', 'Tibiotarsus_length',
       'Tibiotarsus_diameter', 'Tarsometatarsus_length',
       'Tarsometatarsus_diameter',]]
birds_tg = birds[['Species_group_category']]

penguins = pd.read_csv("../datasets/penguins.csv")
print(penguins.shape)
penguins_ft = penguins.loc[:,['Culmen_Length_mm', 'Culmen_Depth_mm', 'Flipper_Length_', 'Body_Massgr', ]]
penguins_tg = penguins[['Species_category']]

#pokemon = pd.read_csv("../datasets/pokemon.csv")

sloths = pd.read_csv("../datasets/sloth_data.csv")
print(sloths.shape)
sloths_ft = sloths.loc[:,['Claw_length_cm', 'Size_cm', 'Tail_length_cm', 'Weight_kg', ]]
sloths_tg = sloths[['Sub_specie_category']]

datasets = [
    (abalones_ft, abalones_tg),
    (birds_ft, birds_tg),
    (penguins_ft, penguins_tg),
    (sloths_ft, sloths_tg)
]

## Define modelos a utilizar


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

classifiers = []

lr = LogisticRegression()
dtc = DecisionTreeClassifier()
svc = SVC()
knn = KNeighborsClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
#classifiers.append(lr)
classifiers.append(dtc)
classifiers.append(svc)
classifiers.append(knn)
classifiers.append(rfc)
classifiers.append(gbc)


## Validación cruzada (Hold Out)


In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,f1_score, roc_auc_score
from imblearn.metrics import sensitivity_score, specificity_score, geometric_mean_score

class DatasetSplitter:
    def __init__(self, test_size, random_state=1111):
        self.test_size = test_size
        self.random_state = random_state
    
    def split_datasets(self, datasets, classifiers):
        results = []
        for i, (data, target) in enumerate(datasets):
            X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=self.test_size, random_state=self.random_state)
            classifier = classifiers
            classifier.fit(X_train, y_train.values.ravel())
            y_pred = classifier.predict(X_test)
            k = len(set(y_test))
            if k==2:
              avg = 'binary'
              multi = 'raise'
            else:
              avg = 'macro'
              multi = 'ovr'

            accuracy = accuracy_score(y_test,y_pred)
            sensitivity = sensitivity_score(y_test,y_pred, average=avg)
            specificity = specificity_score(y_test,y_pred, average=avg)
            f1_sc = f1_score(y_test,y_pred, average=avg)
            gmean = geometric_mean_score(y_test,y_pred, average=avg)
            # auc = roc_auc_score(y_test,y_pred,multi_class=multi)
            results.append((accuracy, sensitivity, specificity, f1_sc, gmean))
        results = pd.DataFrame(results, columns=['accuracy', 'sensitivity', 'specificity', 'f1_sc', 'gmean'])
        return results


In [ ]:
splitter = DatasetSplitter(test_size=0.2, random_state=42)

#for model in classifiers:
#    metrics = splitter.split_datasets(datasets, model)
#    print(model.__class__.__name__)
#    print(metrics)

dtc_metrics = splitter.split_datasets(datasets, classifiers[0])
svc_metrics = splitter.split_datasets(datasets, classifiers[1])
knn_metrics = splitter.split_datasets(datasets, classifiers[2])
rfc_metrics = splitter.split_datasets(datasets, classifiers[3])
gbc_metrics = splitter.split_datasets(datasets, classifiers[4])


In [ ]:
print('Decision Tree Classifier:\n', dtc_metrics)
print('Support Vector Machine:\n', svc_metrics)
print('K Neighbours Classifier:\n', knn_metrics)
print('Random Forest Classifier:\n', rfc_metrics)
print('Gradient Boosting Classifier:\n', gbc_metrics)

In [ ]:
my_dict = {
    'Decision Tree classifier': dtc_metrics,
    'Support Vector Machine Classifier': svc_metrics,
    'K Nearest Neighbour Classifier': knn_metrics,
    'Random Forest Classifier': rfc_metrics,
    'Gradient Boosting Classifier': gbc_metrics
}

In [ ]:
# Plot accuracy metric
for key, value in my_dict.items():
       nth_key = key
       nth_value = value
       plt.plot(nth_value['accuracy'], label=nth_key)
plt.xticks([0, 1, 2, 3, ], ['abalone', 'birds', 'penguins', 'sloths', ],
       rotation=20)
plt.legend()
plt.title('Metric: accuracy_score')


# Prueba de Mann-Whitney para comparar el desempeño de diferentes clasificadores

In [ ]:
from scipy.stats import mannwhitneyu

# Assume we have two classifiers and their accuracy scores in separate lists:
clf1_scores = [0.85, 0.92, 0.88, 0.89, 0.91]
clf2_scores = [0.86, 0.87, 0.83, 0.90, 0.85]

# Perform a Mann-Whitney test to compare the two sets of scores:
u_statistic, p_value = mannwhitneyu(clf1_scores, clf2_scores)

# Print the results:
print(f"U statistic: {u_statistic}")
print(f"P-value: {p_value}")